In [1]:
import os
#先收集所有的样品的浓度
sample_f=open("./sample_template_2nM_fixed.csv")
sample_dict={}
sample_f.readline()
for line in sample_f:
    items=line.strip().split(",")
    project_id=items[0]
    sample_id=items[1]
    sample_concentration=float(items[2])
    well_pos=items[3]
    if sample_concentration<0.3:
        print (line)
        continue
    if project_id in sample_dict.keys():
        sample_dict[project_id][sample_id]=(sample_concentration,well_pos)
    else:
        sample_dict[project_id]={}
        sample_dict[project_id][sample_id]=(sample_concentration,well_pos)
        
#按浓度排序
sample_dict_ordered={}
for key in sample_dict.keys():
    sample_dict_ordered[key]={k:v for k,v in sorted(sample_dict[key].items(),key=lambda item:item[1])}
    


In [2]:
#将样品依据浓度分组到各个组中
pool_dict={}
for key,value in sample_dict_ordered.items():
    tier_list=[]
    min_con=list(value.values())[0][0]
    max_con=list(value.values())[-1][0]
    
    for i in range(0,10):
        if min_con>max_con:
            break
        tier_list.append((min_con,min_con*8))
        min_con=min_con*8
        #print (tier_list[i])

    
    pool_dict[key]={}
    for key1,value1 in value.items():  
        #key1 是sample id，value1 是（sample_con,well_pos）
        #由于要加入well pos 信息，value1 又float 变为了tuple
        sample_concentration=value1[0]
        for range_tuple in tier_list:
            if range_tuple[0] <=sample_concentration<=range_tuple[1]:
                range_name='%s-%s' %(range_tuple[0],range_tuple[1])
                if not range_name in pool_dict[key].keys():
                    pool_dict[key][range_name]={}
                    pool_dict[key][range_name][key1]=value1
                else:
                    pool_dict[key][range_name][key1]=value1
                break
            
        print (key1,value1)
        



1735-FL-1837-01-60 (1.25511808, 'D08')
1735-FL-1837-01-70 (5.76820224, 'F09')
1735-FL-1837-01-74_Cleanup (27.81896114, 'G12')
1735-FL-1837-01-48 (35.14330624, 'H06')
1735-FL-1837-01-89 (39.46608197, 'A12')
1735-FL-1837-01-58 (43.68879104, 'B08')
1735-FL-1837-01-75 (44.38311168, 'C10')
1735-FL-1837-01-49 (46.51948288, 'A07')
1735-FL-1837-01-77 (46.67971072, 'E10')
1735-FL-1837-01-78 (51.64677376, 'F10')
1735-FL-1837-01-61 (61.31385344, 'E08')
1735-FL-1837-01-46 (62.73790995, 'F06')
1735-FL-1837-01-80 (65.5598912, 'H10')
1735-FL-1837-01-26 (66.44114432, 'B04')
1735-FL-1837-01-02 (69.45876864, 'B01')
1735-FL-1837-01-88 (70.2332032, 'H11')
1735-FL-1837-01-62 (71.7019584, 'F08')
1735-FL-1837-01-76 (73.41105536, 'D10')
1735-FL-1837-01-94 (73.49116928, 'F12')
1735-FL-1837-01-40 (76.88265856, 'H05')
1735-FL-1837-01-87 (80.2474432, 'G11')
1735-FL-1837-01-10 (80.88835456, 'B02')
1735-FL-1837-01-85 (83.10483968, 'E11')
1735-FL-1837-01-50 (86.20257792, 'B07')
1735-FL-1837-01-64 (87.1906496, 'H08')

In [3]:
from openpyxl.styles.borders import Border, Side
from openpyxl.styles import Alignment
from openpyxl import Workbook


from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.utils import get_column_letter
pooling_sheet_id=input("pooling id:")
wb_pooling=Workbook()
ws_pooling=wb_pooling.active



#header 部分内容和格式
thin = Side(border_style="thin", color="000000")
ws_pooling.cell(row=1,column=1).value='Pool ID'
ws_pooling.cell(row=1,column=2).value='Sample ID'
ws_pooling.cell(row=1,column=3).value='Well Position'
ws_pooling.cell(row=1,column=4).value='ul to pool'
ws_pooling.cell(row=1,column=5).value='low TE'
for row in ws_pooling['A1:E1']:
    for cell in row:
        cell.border=Border(top=thin,left=thin,right=thin,bottom=thin)
        
        
row_count=2
max_pool_id_size=0
max_sample_id_size=0

sample_dict_update={}
for project_id,value in pool_dict.items():
    pool_number=1
    for range_name,sample_con_dict in value.items():
        pool_id=project_id+'-FL-pool%s' %pool_number
        sample_dict_update[pool_id]=[]
        max_pool_id_size= len(pool_id) if len(pool_id)>max_pool_id_size else max_pool_id_size
        pool_number+=1
        row_start=row_count
        ws_pooling.cell(row=row_start,column=1).value=pool_id
        
        sample_total_volume=0
        sample_con_list=list(sample_con_dict.values())
        sample_counts=len(sample_con_list)
        mid_con=(sample_con_list[0][0]+sample_con_list[-1][0])/2
        total_volume=mid_con*sample_counts
        
        for sample_id in sorted(sample_con_dict):
            if len(sample_id)>max_sample_id_size:
                max_sample_id_size=len(sample_id)
            
            sample_con=sample_con_dict[sample_id][0]
            well_pos=sample_con_dict[sample_id][1]
            pooled_value=2*total_volume/(sample_con*sample_counts)
            sample_total_volume+=pooled_value
            print (sample_id,"%.2f" %sample_con,"%.2f" %pooled_value)
            sample_dict_update[pool_id].append((sample_id,well_pos,pooled_value))

            ws_pooling.cell(row=row_count,column=2).value=sample_id
            ws_pooling.cell(row=row_count,column=3).value=well_pos
            ws_pooling.cell(row=row_count,column=4).value=round(pooled_value,2)
            row_count+=1
        row_end=row_count-1
        print (" ")
        LOW_TE=total_volume-sample_total_volume
        ws_pooling.cell(row=row_start,column=5).value=round(LOW_TE,2)
        #合并单元格
        ws_pooling.merge_cells('A%s:A%s' %(row_start,row_end))
        ws_pooling.merge_cells('E%s:E%s' %(row_start,row_end))
        
        for row in ws_pooling['A%s:E%s' %(row_start,row_end)]:
            for cell in row:
                cell.border=Border(top=thin,left=thin,right=thin,bottom=thin)      
        ws_pooling.cell(row=row_start,column=1).alignment=Alignment(horizontal='center',vertical='center')
        ws_pooling.cell(row=row_start,column=5).alignment=Alignment(horizontal='center',vertical='center')
        row_count+=1
        
ws_pooling.column_dimensions['A'].width = int(max_pool_id_size*1.2)
ws_pooling.column_dimensions['B'].width = int(max_sample_id_size*1.2)
ws_pooling.column_dimensions['C'].width = 15
ws_pooling.column_dimensions['D'].width = 11        
ws_pooling.column_dimensions['E'].width = 11      
wb_pooling.save('%s-pooling.xlsx' %pooling_sheet_id)

pooling id:1735-1837
1735-FL-1837-01-60 1.26 5.60
1735-FL-1837-01-70 5.77 1.22
 
1735-FL-1837-01-02 69.46 1.56
1735-FL-1837-01-26 66.44 1.63
1735-FL-1837-01-40 76.88 1.41
1735-FL-1837-01-46 62.74 1.72
1735-FL-1837-01-48 35.14 3.08
1735-FL-1837-01-49 46.52 2.32
1735-FL-1837-01-58 43.69 2.47
1735-FL-1837-01-61 61.31 1.76
1735-FL-1837-01-62 71.70 1.51
1735-FL-1837-01-74_Cleanup 27.82 3.88
1735-FL-1837-01-75 44.38 2.43
1735-FL-1837-01-76 73.41 1.47
1735-FL-1837-01-77 46.68 2.32
1735-FL-1837-01-78 51.65 2.09
1735-FL-1837-01-80 65.56 1.65
1735-FL-1837-01-87 80.25 1.35
1735-FL-1837-01-88 70.23 1.54
1735-FL-1837-01-89 39.47 2.74
1735-FL-1837-01-94 73.49 1.47
 
1735-FL-1837-01-01 103.64 4.64
1735-FL-1837-01-03 99.80 4.82
1735-FL-1837-01-04 180.55 2.66
1735-FL-1837-01-05 113.60 4.23
1735-FL-1837-01-06 148.66 3.23
1735-FL-1837-01-07 153.04 3.14
1735-FL-1837-01-08 220.18 2.18
1735-FL-1837-01-09 153.77 3.13
1735-FL-1837-01-10 80.89 5.94
1735-FL-1837-01-11 92.32 5.21
1735-FL-1837-01-12 111.17 4.33
1

In [7]:
#自动生成auto pipette 文件
from openpyxl import load_workbook
import os
import csv
auto_pipette_template="./Admera Pooling Template - Final Version.xlsx"
#输出auto pipette的excel 文件
for key,value in sample_dict_update.items():
    wb=load_workbook(auto_pipette_template)
    ws_samples=wb['Samples']
    sample_ID_column=2
    sample_well_column=3
    sample_ul_column=4
    
    row_start=3
    #如果1个plate 中小于5个样品要pool，就手动pool
    if len(value)<5:
        print ('There are manual pools')
        continue
    for item in value:
        
        sample_id=item[0]
        well_pos=item[1]
        add_vol=item[2]
        cell_sample_id=ws_samples.cell(row=row_start,column=sample_ID_column)
        cell_sample_id.value=sample_id
        cell_well_pos=ws_samples.cell(row=row_start,column=sample_well_column)
        cell_well_pos.value=well_pos
        cell_add_vol=ws_samples.cell(row=row_start,column=sample_ul_column)
        cell_add_vol.value=add_vol
        row_start+=1
        
    cell_pool_id=ws_samples.cell(row=3,column=1)
    cell_pool_id.value=key
    auto_excel="%s.xlsx" %key
    wb.save(auto_excel)
    #由于必须用excel存储才会有缓存，因此必须打开，再储存。
    os.startfile(auto_excel)        

There are manual pools


In [5]:
#输出auto pipette 的csv文件
for key,value in sample_dict_update.items():
    row_start=3
    if len(value)<5:
       
        continue
    auto_excel="%s.xlsx" %key
    wb_auto=load_workbook(auto_excel,data_only=True)
    ws_csv=wb_auto['CSV1']
    csv_f=open("./%s.csv" %key,"w+",newline='')
    csv_writer=csv.writer(csv_f,delimiter=",")  
    
    for row in ws_csv.iter_rows():       
        csv_row=[]
        for cell in row:
            #print (cell.value)
            csv_row.append(cell.value)
        csv_writer.writerow(csv_row)
    csv_f.close()
    
    if len(value)>50:
        ws_csv=wb_auto['CSV2']
        csv_f=open("./%s-second.csv" %key,"w+",newline='')
        csv_writer=csv.writer(csv_f,delimiter=",")
        for row in ws_csv.iter_rows():       
            csv_row=[]
            for cell in row:
                #rint (cell.value)
                csv_row.append(cell.value)
            csv_writer.writerow(csv_row)
        csv_f.close()    

In [59]:
round(5.434,2)

5.43